In [13]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 17: Conway Cubes ---</h2><p>As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.</p>
<p>The experimental energy source is based on cutting-edge technology: a set of <span title="Rest in peace, Conway.">Conway</span> Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.</p>
<p>The pocket dimension contains an infinite 3-dimensional grid. At every integer 3-dimensional coordinate (<code>x,y,z</code>), there exists a single cube which is either <em>active</em> or <em>inactive</em>.</p>
<p>In the initial state of the pocket dimension, almost all cubes start <em>inactive</em>. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified <em>active</em> (<code>#</code>) or <em>inactive</em> (<code>.</code>) state.</p>
<p>The energy source then proceeds to boot up by executing six <em>cycles</em>.</p>
<p>Each cube only ever considers its <em>neighbors</em>: any of the 26 other cubes where any of their coordinates differ by at most <code>1</code>. For example, given the cube at <code>x=1,y=2,z=3</code>, its neighbors include the cube at <code>x=2,y=2,z=2</code>, the cube at <code>x=0,y=2,z=3</code>, and so on.</p>
<p>During a cycle, <em>all</em> cubes <em>simultaneously</em> change their state according to the following rules:</p>
<ul>
<li>If a cube is <em>active</em> and <em>exactly <code>2</code> or <code>3</code></em> of its neighbors are also active, the cube remains <em>active</em>. Otherwise, the cube becomes <em>inactive</em>.</li>
<li>If a cube is <em>inactive</em> but <em>exactly <code>3</code></em> of its neighbors are active, the cube becomes <em>active</em>. Otherwise, the cube remains <em>inactive</em>.</li>
</ul>
<p>The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the six-cycle boot process.</p>
<p>For example, consider the following initial state:</p>
<pre><code>.#.
..#
###
</code></pre>
<p>Even though the pocket dimension is 3-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1 region of the 3-dimensional space.)</p>
<p>Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given <code>z</code> coordinate (and the frame of view follows the active cells in each cycle):</p>
<pre><code>Before any cycles:

z=0
.#.
..#

###

After 1 cycle:

z=-1
#..
..#
.#.

z=0
#.#
.##
.#.

z=1
#..
..#
.#.

After 2 cycles:

z=-2
.....
.....
..#..
.....
.....

z=-1
..#..
.#..#
....#
.#...
.....

z=0
##...
##...
#....
....#
.###.

z=1
..#..
.#..#
....#
.#...
.....

z=2
.....
.....
..#..
.....
.....

After 3 cycles:

z=-2
.......
.......
..##...
..###..
.......
.......
.......

z=-1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=0
...#...
.......
#......
.......
.....##
.##.#..
...#...

z=1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=2
.......
.......
..##...
..###..
.......
.......
.......
</code></pre>

<p>After the full six-cycle boot process completes, <em><code>112</code></em> cubes are left in the <em>active</em> state.</p>
<p>Starting with your given initial configuration, simulate six cycles. <em>How many cubes are left in the active state after the sixth cycle?</em></p>
</article>


In [14]:
from copy import deepcopy
from typing import Iterable


example = """
.#.
..#
###
"""


def print_layer(layers: list[list[chr]], layer: int) -> None:
    correction = len(layers) // 2
    print(f"Z={layer-correction}")
    print("\n".join("".join(r[1:-1]) for r in layers[layer][1:-1]))


def print_layers(layers: list[list[chr]]) -> None:
    for layer in range(1, len(layers) - 1):
        print_layer(layers, layer)
        print()


def neigbors(x: int, y: int, z: int) -> Iterable[tuple[int, int, int]]:
    return (
        (z + dz, y + dy, x + dx)
        for dz, dy, dx in product((-1, 0, 1), repeat=3)
        if not (dx == dz == dy == 0)
    )


def active_neigbors(layers: list[list[chr]], x: int, y: int, z: int) -> int:
    return sum(1 for z1, y1, x1 in neigbors(x, y, z) if layers[z1][y1][x1] == "#")


def increase_layer(layers):
    n = len(layers[0]) + 2
    for i in range(len(layers)):
        layers[i] = [["."] * n] + [["."] + r + ["."] for r in layers[i]] + [["."] * n]

    layers.insert(0, [["."] * n for _ in range(n)])
    layers.append([["."] * n for _ in range(n)])


def conway_cubes(s: str, cycles: int = 6, do_print=False) -> int:
    layers = [[list(r) for r in s.strip().splitlines()]]
    if do_print:
        print("Before any cycles")
        print()
    increase_layer(layers)
    if do_print:
        print_layers(layers)

    for cycle in range(1, cycles + 1):
        increase_layer(layers)
        n = len(layers[0])
        m = len(layers)
        new_layers = deepcopy(layers)

        for z, y, x in product(range(1, m - 1), range(1, n - 1), range(1, n - 1)):
            acn = active_neigbors(layers, x, y, z)
            if layers[z][y][x] == "#" and acn not in (2, 3):
                new_layers[z][y][x] = "."
            elif layers[z][y][x] == "." and acn == 3:
                new_layers[z][y][x] = "#"

        layers = new_layers
        if do_print:
            print(f"After {cycle} cycle{'s' if cycle > 1 else ''}:")
            print()
            print_layers(layers)
            print()

    return sum(
        1 for z, y, x in product(range(m), range(n), range(n)) if layers[z][y][x] == "#"
    )


conway_cubes(example)

112

In [15]:
with open("../input/day17.txt") as f:
    puzzle = f.read()

print(f"Part I: {conway_cubes(puzzle)}")

Part I: 289


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>289</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>For some reason, your simulated results don't match what the experimental energy source engineers expected. Apparently, the pocket dimension actually has <em>four spatial dimensions</em>, not three.</p>
<p>The pocket dimension contains an infinite 4-dimensional grid. At every integer 4-dimensional coordinate (<code>x,y,z,w</code>), there exists a single cube (really, a <em>hypercube</em>) which is still either <em>active</em> or <em>inactive</em>.</p>
<p>Each cube only ever considers its <em>neighbors</em>: any of the 80 other cubes where any of their coordinates differ by at most <code>1</code>. For example, given the cube at <code>x=1,y=2,z=3,w=4</code>, its neighbors include the cube at <code>x=2,y=2,z=3,w=3</code>, the cube at <code>x=0,y=2,z=3,w=4</code>, and so on.</p>
<p>The initial state of the pocket dimension still consists of a small flat region of cubes. Furthermore, the same rules for cycle updating still apply: during each cycle, consider the <em>number of active neighbors</em> of each cube.</p>
<p>For example, consider the same initial state as in the example above. Even though the pocket dimension is 4-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1x1 region of the 4-dimensional space.)</p>
<p>Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given <code>z</code> and <code>w</code> coordinate:</p>
<pre><code>Before any cycles:

z=0, w=0
.#.
..#

###

After 1 cycle:

z=-1, w=-1
#..
..#
.#.

z=0, w=-1
#..
..#
.#.

z=1, w=-1
#..
..#
.#.

z=-1, w=0
#..
..#
.#.

z=0, w=0
#.#
.##
.#.

z=1, w=0
#..
..#
.#.

z=-1, w=1
#..
..#
.#.

z=0, w=1
#..
..#
.#.

z=1, w=1
#..
..#
.#.

After 2 cycles:

z=-2, w=-2
.....
.....
..#..
.....
.....

z=-1, w=-2
.....
.....
.....
.....
.....

z=0, w=-2
###..
##.##
#...#
.#..#
.###.

z=1, w=-2
.....
.....
.....
.....
.....

z=2, w=-2
.....
.....
..#..
.....
.....

z=-2, w=-1
.....
.....
.....
.....
.....

z=-1, w=-1
.....
.....
.....
.....
.....

z=0, w=-1
.....
.....
.....
.....
.....

z=1, w=-1
.....
.....
.....
.....
.....

z=2, w=-1
.....
.....
.....
.....
.....

z=-2, w=0
###..
##.##
#...#
.#..#
.###.

z=-1, w=0
.....
.....
.....
.....
.....

z=0, w=0
.....
.....
.....
.....
.....

z=1, w=0
.....
.....
.....
.....
.....

z=2, w=0
###..
##.##
#...#
.#..#
.###.

z=-2, w=1
.....
.....
.....
.....
.....

z=-1, w=1
.....
.....
.....
.....
.....

z=0, w=1
.....
.....
.....
.....
.....

z=1, w=1
.....
.....
.....
.....
.....

z=2, w=1
.....
.....
.....
.....
.....

z=-2, w=2
.....
.....
..#..
.....
.....

z=-1, w=2
.....
.....
.....
.....
.....

z=0, w=2
###..
##.##
#...#
.#..#
.###.

z=1, w=2
.....
.....
.....
.....
.....

z=2, w=2
.....
.....
..#..
.....
.....
</code></pre>

<p>After the full six-cycle boot process completes, <em><code>848</code></em> cubes are left in the <em>active</em> state.</p>
<p>Starting with your given initial configuration, simulate six cycles in a 4-dimensional space. <em>How many cubes are left in the active state after the sixth cycle?</em></p>
</article>

</main>


In [16]:
from copy import deepcopy
from typing import Iterable


example = """
.#.
..#
###
"""


def print_layer(layers: list[list[chr]], w: int, z: int) -> None:
    w_correction = len(layers) // 2
    z_correction = len(layers[0]) // 2
    print(f"z={z-z_correction}, w={w-w_correction}")
    print("\n".join("".join(r[1:-1]) for r in layers[w][z][1:-1]))


def print_layers(layers: list[list[chr]]) -> None:
    for w in range(1, len(layers) - 1):
        for z in range(1, len(layers[0]) - 1):
            print_layer(layers, w, z)
            print()


def neigbors(x: int, y: int, z: int, w: int) -> Iterable[tuple[int, int, int, int]]:
    return (
        (w + dw, z + dz, y + dy, x + dx)
        for dw, dz, dy, dx in product((-1, 0, 1), repeat=4)
        if not (dw == dz == dy == dx == 0)
    )


def active_neigbors(layers: list[list[chr]], x: int, y: int, z: int, w: int) -> int:
    return sum(
        1 for w1, z1, y1, x1 in neigbors(x, y, z, w) if layers[w1][z1][y1][x1] == "#"
    )


def increase_layer(layers):
    n = len(layers[0][0]) + 2
    m = len(layers[0]) + 2
    for w in range(len(layers)):
        for z in range(len(layers[w])):
            layers[w][z] = (
                [["."] * n] + [["."] + r + ["."] for r in layers[w][z]] + [["."] * n]
            )

        layers[w].insert(0, [["."] * n for _ in range(n)])
        layers[w].append([["."] * n for _ in range(n)])

    layers.insert(0, [[["."] * n for _ in range(n)] for _ in range(m)])
    layers.append([[["."] * n for _ in range(n)] for _ in range(m)])


def conway_cubes(s: str, cycles: int = 6, do_print=False) -> int:
    layers = [[[list(r) for r in s.strip().splitlines()]]]
    if do_print:
        print("Before any cycles")
        print()

    increase_layer(layers)

    if do_print:
        print_layers(layers)

    n = len(layers[0][0])
    m = len(layers[0])
    o = len(layers)

    for cycle in range(1, cycles + 1):
        increase_layer(layers)
        n = len(layers[0][0])
        m = len(layers[0])
        o = len(layers)
        new_layers = deepcopy(layers)

        for w, z, y, x in product(
            range(1, o - 1), range(1, m - 1), range(1, n - 1), range(1, n - 1)
        ):
            acn = active_neigbors(layers, x, y, z, w)
            if layers[w][z][y][x] == "#" and acn not in (2, 3):
                new_layers[w][z][y][x] = "."
            elif layers[w][z][y][x] == "." and acn == 3:
                new_layers[w][z][y][x] = "#"

        layers = new_layers
        if do_print:
            print(f"After {cycle} cycle{'s' if cycle > 1 else ''}:")
            print()
            print_layers(layers)
            print()

    return sum(
        1
        for w, z, y, x in product(range(o), range(m), range(n), range(n))
        if layers[w][z][y][x] == "#"
    )


assert conway_cubes(example) == 848

In [17]:
print(f"Part II: {conway_cubes(puzzle)}")

Part II: 2084


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>2084</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
